In [58]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("varunraskar/cancer-regression")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cancer-regression


In [60]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
dataframe = pd.read_csv('/kaggle/input/cancer-regression/cancer_reg.csv', sep=',')

dataframe.shape

(3047, 33)

In [63]:
dataframe.head()

,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",39.3,...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",33.0,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",45.0,...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",42.8,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",48.3,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657


In [62]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avganncount              3047 non-null   float64
 1   avgdeathsperyear         3047 non-null   int64  
 2   target_deathrate         3047 non-null   float64
 3   incidencerate            3047 non-null   float64
 4   medincome                3047 non-null   int64  
 5   popest2015               3047 non-null   int64  
 6   povertypercent           3047 non-null   float64
 7   studypercap              3047 non-null   float64
 8   binnedinc                3047 non-null   object 
 9   medianage                3047 non-null   float64
 10  medianagemale            3047 non-null   float64
 11  medianagefemale          3047 non-null   float64
 12  geography                3047 non-null   object 
 13  percentmarried           3047 non-null   float64
 14  pctnohs18_24            

In [64]:
dataframe.dtypes

,0
avganncount,float64
avgdeathsperyear,int64
target_deathrate,float64
incidencerate,float64
medincome,int64
popest2015,int64
povertypercent,float64
studypercap,float64
binnedinc,object
medianage,float64


In [65]:
dataframe.describe()

,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,medianage,medianagemale,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
count,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3.047000e+03,3047.000000,3047.000000,3047.000000,3047.000000,...,2438.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000
mean,606.338544,185.965868,178.664063,448.268586,47063.281917,1.026374e+05,16.878175,155.399415,45.272333,39.570725,...,48.453774,41.196324,36.252642,19.240072,83.645286,9.107978,1.253965,1.983523,51.243872,5.640306
std,1416.356223,504.134286,27.751511,54.560733,12040.090836,3.290592e+05,6.409087,529.628366,45.304480,5.226017,...,10.083006,9.447687,7.841741,6.113041,16.380025,14.534538,2.610276,3.517710,6.572814,1.985816
min,6.000000,3.000000,59.700000,201.300000,22640.000000,8.270000e+02,3.200000,0.000000,22.300000,22.400000,...,15.700000,13.500000,11.200000,2.600000,10.199155,0.000000,0.000000,0.000000,22.992490,0.000000
25%,76.000000,28.000000,161.200000,420.300000,38882.500000,1.168400e+04,12.150000,0.000000,37.700000,36.350000,...,41.000000,34.500000,30.900000,14.850000,77.296180,0.620675,0.254199,0.295172,47.763063,4.521419
50%,171.000000,61.000000,178.100000,453.549422,45207.000000,2.664300e+04,15.900000,0.000000,41.000000,39.600000,...,48.700000,41.100000,36.300000,18.800000,90.059774,2.247576,0.549812,0.826185,51.669941,5.381478
75%,518.000000,149.000000,195.200000,480.850000,52492.000000,6.867100e+04,20.400000,83.650776,44.000000,42.500000,...,55.600000,47.700000,41.550000,23.100000,95.451693,10.509732,1.221037,2.177960,55.395132,6.493677
max,38150.000000,14010.000000,362.800000,1206.900000,125635.000000,1.017029e+07,47.400000,9762.308998,624.000000,64.700000,...,78.900000,70.700000,65.100000,46.600000,100.000000,85.947799,42.619425,41.930251,78.075397,21.326165


In [84]:
target = 'target_deathrate'

In [83]:
features = [
    'avganncount',
    'avgdeathsperyear',
    'incidencerate',
    'medincome',
    'povertypercent',
    'studypercap',
    'binnedinc',
    'medianage',
    'medianagemale',
    'medianagefemale',
    'percentmarried',
    'pctmarriedhouseholds',
    'pctnohs18_24',
    'pcths18_24',
    'pctsomecol18_24',
    'pctbachdeg18_24',
    'pcths25_over',
    'pctbachdeg25_over',
    'pctemployed16_over',
    'pctunemployed16_over',
    'pctprivatecoverage',
    'pctprivatecoveragealone',
    'pctempprivcoverage',
    'pctpubliccoverage',
    'pctpubliccoveragealone',
    'pctwhite',
    'pctblack',
    'pctasian',
    'pctotherrace',
    'birthrate'
]

print(features)

['avganncount', 'avgdeathsperyear', 'incidencerate', 'medincome', 'povertypercent', 'studypercap', 'binnedinc', 'medianage', 'medianagemale', 'medianagefemale', 'percentmarried', 'pctmarriedhouseholds', 'pctnohs18_24', 'pcths18_24', 'pctsomecol18_24', 'pctbachdeg18_24', 'pcths25_over', 'pctbachdeg25_over', 'pctemployed16_over', 'pctunemployed16_over', 'pctprivatecoverage', 'pctprivatecoveragealone', 'pctempprivcoverage', 'pctpubliccoverage', 'pctpubliccoveragealone', 'pctwhite', 'pctblack', 'pctasian', 'pctotherrace', 'birthrate']


In [69]:
dataframe.isnull().sum()

,0
avganncount,0
avgdeathsperyear,0
target_deathrate,0
incidencerate,0
medincome,0
popest2015,0
povertypercent,0
studypercap,0
binnedinc,0
medianage,0


# Task
Input missing data using KNN imputation.

## Identify columns for imputation

### Subtask:
List the columns that have missing values and are suitable for KNN imputation.


**Reasoning**:
Based on the previous output of `dataframe.isnull().sum()` and `dataframe.dtypes`, identify the numerical columns with missing values suitable for KNN imputation.



In [70]:
#Código gerado pelo Gemini
columns_with_missing_values = dataframe.isnull().sum()
columns_with_missing_values = columns_with_missing_values[columns_with_missing_values > 0].index.tolist()

numerical_columns = dataframe.select_dtypes(include=['float64', 'int64']).columns.tolist()

columns_for_imputation = [col for col in columns_with_missing_values if col in numerical_columns]

print(columns_for_imputation)

['pctsomecol18_24', 'pctemployed16_over', 'pctprivatecoveragealone']


**Reasoning**:
Create a new dataframe containing only the columns listed in the `columns_for_imputation` variable from the `dataframe`.



In [71]:
dataframe_for_imputation = dataframe[columns_for_imputation].copy()
display(dataframe_for_imputation.head())

,pctsomecol18_24,pctemployed16_over,pctprivatecoveragealone
0,42.1,51.9,NaN
1,64.0,55.9,53.8
2,NaN,45.9,43.5
3,36.1,48.3,40.3
4,40.0,48.2,43.9


## Apply knn imputer

### Subtask:
Apply the KNNImputer on the selected numerical features.


**Reasoning**:
Apply the KNNImputer on the selected numerical features.



In [72]:
#Código gerado pelo Gemini
from sklearn.impute import KNNImputer

imputer = KNNImputer()

imputed_data = imputer.fit_transform(dataframe_for_imputation)

print(imputed_data)

[[42.1  51.9  42.3 ]
 [64.   55.9  53.8 ]
 [38.66 45.9  43.5 ]
 ...
 [33.3  64.8  52.6 ]
 [47.96 59.3  56.3 ]
 [43.9  48.8  60.3 ]]


**Reasoning**:
Convert the imputed NumPy array back to a DataFrame, drop the original columns with missing values from the main DataFrame, and then merge the imputed data back into the main DataFrame.



In [75]:
#Código gerado pelo Gemini
imputed_dataframe = pd.DataFrame(imputed_data, columns=columns_for_imputation)

dataframe = dataframe.drop(columns=columns_for_imputation)

dataframe = pd.merge(dataframe, imputed_dataframe, left_index=True, right_index=True)

display(dataframe.head())

,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,pctsomecol18_24,pctemployed16_over,pctprivatecoveragealone
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",39.3,...,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831,42.10,51.9,42.3
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",33.0,...,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096,64.00,55.9,53.8
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",45.0,...,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488,38.66,45.9,43.5
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",42.8,...,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841,36.10,48.3,40.3
4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",48.3,...,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657,40.00,48.2,43.9


In [79]:
from sklearn.preprocessing import RobustScaler

numerical_features_to_scale = dataframe.select_dtypes(include=['float64', 'int64']).columns.tolist()
numerical_features_to_scale.remove(target)

robust_scaler = RobustScaler()

dataframe[numerical_features_to_scale] = robust_scaler.fit_transform(dataframe[numerical_features_to_scale])

dataframe.info()

,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,medianage,...,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,pctsomecol18_24,pctemployed16_over,pctprivatecoveragealone
0,2.773756,3.371901,164.9,0.598688,1.226423,4.097215,-0.569697,5.974221,"(61494.5, 125635]",-0.269841,...,-0.581818,-0.456018,0.035105,4.418573,0.540312,0.155415,0.373863,0.177156,-0.219913,-0.429133
1,0.004525,0.074380,161.3,-0.692806,0.214556,0.291751,0.327273,0.276282,"(48021.6, 51046.4]",-1.269841,...,-0.424242,-0.045786,-0.129282,1.754607,1.548324,-0.825129,-0.531565,2.729604,0.126407,0.372819
2,-0.156109,-0.090909,174.7,-1.715102,0.304273,-0.098566,-0.157576,0.568556,"(48021.6, 51046.4]",0.634921,...,0.278788,0.047502,-0.152482,-0.086792,1.020387,0.363588,-0.837614,-0.223776,-0.739394,-0.345451
3,0.579186,1.165289,194.8,-0.382319,-0.070833,0.864039,0.145455,4.096044,"(42724.4, 45201]",0.285714,...,0.751515,0.092804,-0.148139,0.632523,0.284927,-0.084961,-0.394288,-0.522145,-0.531602,-0.568603
4,-0.257919,-0.289256,144.4,-1.708496,0.348874,-0.286416,-0.412121,0.000000,"(48021.6, 51046.4]",1.158730,...,0.472727,0.222756,-0.199957,0.119998,-0.177423,0.308896,0.717543,-0.067599,-0.540260,-0.317557



Scaled DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avganncount              3047 non-null   float64
 1   avgdeathsperyear         3047 non-null   float64
 2   target_deathrate         3047 non-null   float64
 3   incidencerate            3047 non-null   float64
 4   medincome                3047 non-null   float64
 5   popest2015               3047 non-null   float64
 6   povertypercent           3047 non-null   float64
 7   studypercap              3047 non-null   float64
 8   binnedinc                3047 non-null   object 
 9   medianage                3047 non-null   float64
 10  medianagemale            3047 non-null   float64
 11  medianagefemale          3047 non-null   float64
 12  geography                3047 non-null   object 
 13  percentmarried           3047 non-null   float64
 14  

In [93]:
# Identify object columns to encode
object_columns = dataframe.select_dtypes(include=['object']).columns.tolist()

# Apply One-Hot Encoding
dataframe = pd.get_dummies(dataframe, columns=object_columns, drop_first=True)

# Display the first few rows of the dataframe after encoding

print(dataframe.dtypes)

avganncount                                    float64
avgdeathsperyear                               float64
target_deathrate                               float64
incidencerate                                  float64
medincome                                      float64
                                                ...   
geography_Yukon-Koyukuk Census Area, Alaska       bool
geography_Yuma County, Arizona                    bool
geography_Yuma County, Colorado                   bool
geography_Zapata County, Texas                    bool
geography_Zavala County, Texas                    bool
Length: 3086, dtype: object


In [95]:
linear_regression_model = LinearRegression()
print(dataframe.dtypes)

# Update features to include all columns except the target after one-hot encoding
X = dataframe.drop(columns=[target])
Y = dataframe[target]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("\nShape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

avganncount                                    float64
avgdeathsperyear                               float64
target_deathrate                               float64
incidencerate                                  float64
medincome                                      float64
                                                ...   
geography_Yukon-Koyukuk Census Area, Alaska       bool
geography_Yuma County, Arizona                    bool
geography_Yuma County, Colorado                   bool
geography_Zapata County, Texas                    bool
geography_Zavala County, Texas                    bool
Length: 3086, dtype: object

Shape of X_train: (2437, 3085)
Shape of X_test: (610, 3085)
Shape of y_train: (2437,)
Shape of y_test: (610,)


In [96]:
linear_regression_model.fit(X_train, y_train)

LinearRegression()

In [97]:
cv_scores = cross_val_score(linear_regression_model, X, Y, cv=5, scoring='r2')

In [98]:
print(f"R² médio na validação cruzada: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

R² médio na validação cruzada: 0.4747 ± 0.0456


In [100]:
# Fazer previsões no conjunto de teste
y_pred = linear_regression_model.predict(X_test)

# Calcular as métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R2): {r2:.4f}")

Mean Absolute Error (MAE): 15.1761
Mean Squared Error (MSE): 418.0328
Root Mean Squared Error (RMSE): 418.0328
R-squared (R2): 0.4891


In [101]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)

In [102]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [103]:
cv_scores = cross_val_score(rf, X, Y, cv=5, scoring='r2')

In [104]:
print(f"R² médio na validação cruzada: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

R² médio na validação cruzada: 0.5121 ± 0.0431


In [105]:
# Fazer previsões no conjunto de teste
y_pred = rf.predict(X_test)

# Calcular as métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R2): {r2:.4f}")

Mean Absolute Error (MAE): 14.1443
Mean Squared Error (MSE): 371.6271
Root Mean Squared Error (RMSE): 371.6271
R-squared (R2): 0.5458


In [106]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha=1.0)

In [107]:
ridge.fit(X_train, y_train)

Ridge()

In [108]:
# Fazer previsões no conjunto de teste
y_pred = ridge.predict(X_test)

# Calcular as métricas de avaliação
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R2): {r2:.4f}")

Mean Absolute Error (MAE): 15.1065
Mean Squared Error (MSE): 413.3788
Root Mean Squared Error (RMSE): 413.3788
R-squared (R2): 0.4948
